In [1]:
!pip install pyrsgis

In [0]:
######################### Import des libraries #######################

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pyrsgis import raster
from torch.utils.data import Dataset, DataLoader, sampler
from pyrsgis.convert import changeDimension
from copy import deepcopy
import torch
import torch.nn as nn
import pandas as pd
import scipy.signal
import scipy.ndimage
from os import path
import numpy as np
from time import time
from torchvision import transforms
import random
from copy import deepcopy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
############################## Parameters ############################

stop = 1000
epochs = 7

In [0]:
######################### Import des données #########################

path = "/content/drive/My Drive/D4G/2_Seredou_2017/"

Test = "2_Image_Seredou_32bits_20170205/S2A_20170205_seredou_32bits.tif"
MSI = "2_MSI_Seredou_20170205/S2A_20170205_seredou_ZE_MSI_89.tif"
Truth = "GroundTruth_Seredou_20170205/GroundTruth_20170205_seredou_ZE_89.tif"

In [0]:
######################### Fonctions auxiliaires #########################

def divide_test_in_squares(size, img):
    return [img[:,int(i % img.shape[0])*size:((int(i % img.shape[0])+1)*size), int(i // img.shape[0])*size:(int(i // img.shape[0])+1)*size] for i in range(int(img.shape[0] * img.shape[1] // size**2))]

def divide_image_in_squares(size, img):
    return [img[int(i % img.shape[0])*size:((int(i % img.shape[0])+1)*size), int(i // img.shape[0])*size:(int(i // img.shape[0])+1)*size] for i in range(int(img.shape[0] * img.shape[1] // size**2))]

def get_window(img, center, size):
    
    window = torch.tensor([[0. for i in range(size)] for j in range(size)])
    if center[0] + size // 2 < img.shape[0]:
        i_begin = max(0,int(center[0]-size//2))
        i_end = i_begin + size
    else:
        i_end = img.shape[0]
        i_begin = i_end - size
    if center[1] + size // 2 < img.shape[1]:
        j_begin = max(0,int(center[1]-size//2))
        j_end = j_begin + size
    else:
        j_end = img.shape[1]
        j_begin = j_end - size

    for i in range(i_begin, i_end):
        for j in range(j_begin, j_end):
            window[i - i_begin,j - j_begin] = float(img[i,j])
    return window

In [0]:
######################### Base de données #########################

class OurDataset(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, Test, MSI, Truth, size, percentage, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        #get data
        ds1, data_MSI = raster.read(path + MSI, bands='all')
        ds2, data_test = raster.read(path + Test, bands='all')
        ds3, data_truth = raster.read(path + Truth, bands='all')

        data_MSI[data_MSI < 0] = 0
        data_test[data_test < 0] = 0
        data_truth[data_truth < 0] = 0

        self.MSI = divide_image_in_squares(size, data_MSI)
        self.Test = divide_test_in_squares(size, data_test)
        self.Truth = divide_image_in_squares(size, data_truth)
  
        self.size = size
        self.mode = "train"
        self.id_train, self.id_test = train_test_split([i for i in range(len(self.Test))], test_size = percentage, random_state=42, shuffle=True)
        
    def __len__(self):
        if self.mode == "train":
            return len(self.id_train)
        else:
            return len(self.id_test)

    def __getitem__(self, id):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 11 channels value
                mask: true value
        """
        if self.mode == "train":
            idx = self.id_train[id]
        else:
            idx = self.id_test[id]

        feature_data = torch.tensor([[[0. for i in range(11*5)] for j in range(self.Test[idx].shape[0])] for k in range(self.Test[idx].shape[1])])
        feature_truth = np.array([[0 for j in range(self.Test[idx].shape[0])] for k in range(self.Test[idx].shape[1])])
        
        #On importe les données
        for i in range(self.Test[idx].shape[0]):
            for j in range(self.Test[idx].shape[1]):
                
                feature_truth[i,j] = self.Truth[idx][i,j]

                msi = get_window(self.MSI[idx], (i,j), self.size // 2)
                msi = msi/msi.max()
                feature_data[i,j,50] = float(self.MSI[idx][i,j])
                feature_data[i,j,51] = msi.max()
                feature_data[i,j,52] = msi.min()
                feature_data[i,j,53] = msi.mean()
                feature_data[i,j,54] = msi.std()
                
                for k in range (10):
                    test = get_window(self.Test[idx][k,:,:], (i,j), self.size // 2)
                    test = test/test.max()
                    
                    feature_data[i,j,k*5] = float(self.Test[idx][k,i,j])
                    feature_data[i,j,k*5 + 1] = test.max()
                    feature_data[i,j,k*5 + 2] = test.min()
                    feature_data[i,j,k*5 + 3] = test.mean()
                    feature_data[i,j,k*5 + 4] = test.std()
        
        sample = {'data': feature_data, 'mask': feature_truth}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def set_mode(self, mode):
        """Change mode of the database"""
        self.mode = mode

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
def get_activation(activation_type):
    activation_type = activation_type.lower()
    if hasattr(nn, activation_type):
      return getattr(nn, activation_type)()
    else:
      return nn.ReLU()

def _make_nConv(in_channels, out_channels, nb_Conv, activation='ReLU'):
  layers = []
  layers.append(ConvBatchNorm(in_channels, out_channels, activation))

  for _ in range(nb_Conv-1):
      layers.append(ConvBatchNorm(out_channels, out_channels, activation))
  return nn.Sequential(*layers)

class ConvBatchNorm(nn.Module):
  """(convolution => [BN] => ReLU)"""
  
  def __init__(self, in_channels, out_channels, activation='ReLU'):
    super(ConvBatchNorm, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, 
                          kernel_size=3, padding=1)
    self.norm = nn.BatchNorm2d(out_channels)
    self.activation = get_activation(activation)
      
  def forward(self, x):
    out = self.conv(x)
    out = self.norm(out)
    return self.activation(out)

class DownBlock(nn.Module):
  """Downscaling with maxpool convolution"""

  def __init__(self, in_channels, out_channels, nb_Conv, activation='ReLU'):
    super(DownBlock, self).__init__()
    self.maxpool = nn.MaxPool2d(2)
    self.nConvs = _make_nConv(in_channels, out_channels, nb_Conv, activation)
        
  def forward(self, x):
    out = self.maxpool(x)
    return self.nConvs(out)  

class UpBlock(nn.Module):
  """Upscaling then conv"""

  def __init__(self, in_channels, out_channels, nb_Conv, in_padding=0, out_padding=1, activation='ReLU'):
    super(UpBlock, self).__init__()
    self.up = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2,\
                                 padding=in_padding, output_padding=out_padding)
    self.nConvs = _make_nConv(in_channels, out_channels, nb_Conv, activation)

  def forward(self, x, skip_x):
    out = self.up(x)
    x = torch.cat([out, skip_x], dim=1) 
    return self.nConvs(x)

In [0]:
class UNet(nn.Module):
  def __init__(self, n_channels, n_classes):
    '''
    n_channels : number of channels of the input. 
                    By default 4, because we have 4 modalities
    n_labels : number of channels of the ouput.
                  By default 4 (3 labels + 1 for the background)
    '''
    super(UNet, self).__init__()
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.inc = ConvBatchNorm(n_channels, 64)
    self.down1 = DownBlock(64, 128, nb_Conv=2)
    self.down2 = DownBlock(128, 256, nb_Conv=2)
    self.down3 = DownBlock(256, 512, nb_Conv=2)
    self.up1 = UpBlock(512+256, 256, nb_Conv=2, in_padding=0, out_padding=0)
    self.up2 = UpBlock(256+128, 128, nb_Conv=2)
    self.up3 = UpBlock(128+64, 64, nb_Conv=2, in_padding=0, out_padding=0)
    self.outc = nn.Conv2d(64, n_classes, kernel_size=3, stride=1, padding=1)
    self.last_activation = get_activation('Sigmoid')
  
  def forward(self, x):
    x1 = self.inc(x)
    x2 = self.down1(x1)
    x3 = self.down2(x2)
    x4 = self.down3(x3)
    x = self.up1(x4, x3)
    x = self.up2(x, x2)
    x = self.up3(x, x1)
    logits = self.last_activation(self.outc(x))
    return logits

In [0]:
def train(model, train_loader, criterion, optimizer, n_epochs):
    """
    Method used to train a nn
    
    Args:
        model: (nn.Module) the neural network
        train_loader: (DataLoader) a DataLoader wrapping the dataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
        optimizer: (torch.optim) an optimization algorithm
        n_epochs: (int) number of epochs performed during training

    Returns:
        best_model: (nn.Module) the trained neural network
    """
    best_model = deepcopy(model)
    train_best_loss = np.inf

    batch_size = train_loader.batch_size
    n = 10

    n_batches = n//batch_size

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        for i, data in enumerate(train_loader):
            images, mask = data['data'], data['mask']
            scans = torch.flatten(mask)
            outputs = torch.reshape(model(images), (scans.shape[0],4))
            loss = criterion(outputs, scans)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            optimizer.zero_grad()

        mean_loss = total_loss / len(train_loader.dataset)
        print('Epoch %i: loss = %f & accuracy = %f' % (epoch, mean_loss,torch.sum(torch.argmax(outputs,1) == scans)/scans.shape[0]))

        if mean_loss < train_best_loss:
            best_model = deepcopy(model)
            train_best_loss = mean_loss
    
    return best_model

def test(model, data_loader, criterion):
    """
    Method used to test a CNN
    
    Args:
        model: (nn.Module) the neural network
        data_loader: (DataLoader) a DataLoader wrapping a MRIDataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
    """
    model.eval()

    batch_size = data_loader.batch_size
    n = 10000

    n_batches = n//batch_size
    nb_true = 0
    nb_total = 0
    size_loss = 0
    total_loss = 0
    
    with torch.no_grad():
        for i, data in enumerate(data_loader):
             images, mask = data['data'], data['mask']
             scans = torch.flatten(mask)
             outputs = torch.reshape(model(images), (scans.shape[0],4))
             loss = criterion(outputs, scans)
             total_loss += loss.item()
             size_loss += batch_size
             nb_true += int(torch.sum(torch.argmax(outputs,1) == scans))
             nb_total += scans.shape[0]

    print("Final loss : {} & accuracy : {}".format(str(total_loss/ size_loss), str(nb_true/nb_total)))

In [0]:
########################## Database ############################

database = OurDataset(path, Test, MSI, Truth, 10, 0.2)

In [0]:
########################## Prétraitement ############################

batch_size = 10

database.set_mode("train")
dataloader_train = DataLoader(database, batch_size=batch_size, drop_last=True)

model = UNet(n_channels=10, n_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
n_epochs = 5

In [0]:
#################### Entrainement du modèle ##########################

train(model, dataloader_train, criterion, optimizer, n_epochs)
torch.save(model.state_dict(), "model")

In [0]:
############################### Statistiques ################################

#Predict for test data

database.set_mode("test")
dataloader_test = DataLoader(database, batch_size=batch_size, drop_last=True)

test(model, dataloader_test, criterion)